In [1]:
import pandas as pd
import numpy as np
import random

## Set globals 

Definition of each row: brand-month level.  



In [2]:
master_market_list = ["Total US"]
master_retailer_list = ["All Outlets"]
master_dpt_list = ["BEVERAGES"]

### Define Column Definitions

$ is defined as sales. Since this somehow is a special character, I will call that sales below. 

Assumptions:
1. DONE: Start out with Units. Randomly generate.    Coffee ~ N(2000, 200), KSSB ~ N(1000, 50), Powerdered ~ N(500, 50).  
2. DONE: Assume that all brands will have packages sizes = {small, large}.  Define a latent variable package_ratio = percentage of small packages.  For simplicity, let's do price as 7.50 for small and 9.99 for large, for all three categories for now.  
    - Coffee: small = 0.6 lb, large = 1 lb.  package_ratio = 0.5.   
    - KSSB: small = 1lb, large 2 lb.  package_ratio = 1.
    - Powered: small = 1lb, large 2 lb. package_ratio = 1
 
 
3. DONE :AVG_ITEMS: average number of items per store.  \~ N+(5, 3).  This assumption is true across brands and retailers.
4. DONE: TDP = %ACV * AVG_ITEMS. 
5. DONE: Feat & Disp Sales is usually the most expensive, make this the lowest.  
6. DONE: Other columns such as Feat w/o Disp Sales, Disp w/o Feat Sales, and Prc Desc Sales, are about the same. 
7. DONE: No Promo Sales versus Promo Sales:  At the monthly level, can vary quite a bit, but at the annual level, usually 60 \% vs. 40 \%.  
8. NOTE: Promo Sales = Prc Desc Sales + Disp w/o Feat Sales + Feat & Disp Sales + Feat w/o Disp Sales. 
9. DONE: Make sure for Honest Kids, Motts, and Good 2 Grow, the sales and units are increased for this year versus YA (year ago). 


#### Let's use the Teaching Story to anchor our data generating process:

Focal brand: Hawaiian Punch.  

1. DONE: Make sure the $ for KSSB are lower than the other categories, for the current period.

2. DONE: Units are down by 4\%:  Units summed for the Hawaiian Punch is 4 \% lower than Units_YA.  

3. THIS DOESN'T MAKE SENSE IF WE ALSO WANT BELOW: Divide dollars by units, and you can create the volume price: make sure that Price is 1\% lower than the Price\_YA for the brand.   

4. DONE: Make sure that Hawaiian Punch \\$ is 6 percent down from \$_YA. 




In [3]:
all_dates = ["06/03/17","07/01/17","08/05/17","09/02/17","09/30/17","11/04/17","12/02/17","12/30/17"
         ,"02/03/18","03/03/18","03/31/18","05/05/18"]

cols = {"names" : ["Market", "Retailer", "Department", "Category", "Brand", "Period (4we)"
                   , "$", "Lbs", "Units", "Avg_Items", "%ACV", "TDP"
                   , "No Promo$", "Prc Desc $", "Disp w/o Feat $", "Feat & Disp $", "Feat w/o Disp $"
                  ],
        "type" : ["str", "str", "str","str", "str", "str"
                 , "num", "num", "num", "num", "perc", "perc"
                 , "num", "num", "num", "num", "num"]
       }

category_brand_dic = [ {
            "Category" : "DRY COFFEE",
            "package_ratio" : 0.6,
            "price" : [7.50, 9.99],
            "lb" : [0.6, 1],
            "Brand" : ["BARNIE'S","C D M","CAFE BUSTELO","CAFE ESCAPES","CAFE PILON","CAIN'S"
                       ,"CAMERON'S","CARIBOU COFFEE","CHAMELEON", "CHOCK FULL O NUTS","COFFEE PEOPLE","COMMUNITY"
                       ,"DON FRANCISCO","DONUT HOUSE","DUNKIN DONUTS (SMUCKERS)","EIGHT O'CLOCK", "EMERIL'S"
                       ,"FOLGER'S","GLORIA JEAN'S","GREEN MOUNTAIN","GROVE SQUARE","HILLS BROS","ILLY"
                       ,"J F G","LAVAZZA","MARTINSON","MEDAGLIA D'ORO","MELITTA","MJB","NESCAFE","NEW ENGLAND"
                       ,"NEWMAN'S OWN ORGANICS","PAPANICHOLAS","PEET'S COFFEE","SAN FRANCISCO BAY"
                       ,"SEATTLE'S BEST","STARBUCKS","TULLY'S","VAN HOUTTE","GEVALIA","MAX BY MAXWELL HOUSE"
                       ,"MAXWELL HOUSE","MC CAFE","TASSIMO","YUBAN","RET BR"]},
        {
            "Category" : "KIDS SINGLE SERVE BEVERAGES",
            "package_ratio" : 1.0,
            "price" : [7.50, 9.99],
            "lb" : [1, 2],
            "Brand" : ["ALPINA","AO CADBURY BRND","AO COCA-COLA BRND","AO JEL SERT BRND","AO NATIONAL BEVERAGE BRND"
                       ,"AO NESTLE BRND","AO PEPSICO BRND","AO SUNSWEET GROWERS BRND","APPLE & EVE","ARIZONA"
                       ,"BACK TO NATURE","BLUE DIAMOND","BLUEBIRD","BUG JUICE","CHAOKOH","CHERIBUDDY","CHERIBUNDI"
                       ,"CONCHITA","CONCORD FOODS","COUNTRY TIME","DEL FRUTAL","DELAWARE PUNCH","DOLE","DONALD DUCK"
                       ,"EARTH'S BEST","EL MEXICANO","FOCO","GOOD 2 GROW","GOYA","HANSEN'S","HAPPY DRINKS"
                       ,"HAWAIIAN PUNCH","HERSHEY'S CO HERSHEY","HI-C","HOGWASH","HONEST KIDS","HORIZON","ICEE"
                       ,"ITALIA GARDEN","JELLEY BROWN","JUICY JUICE","JUMEX","KERNS","LANGERS","LIBBY'S","LIPTON"
                       ,"LITTLE HUG","MARTINELLI'S","MINUTE MAID","MONDO","MOTTS","OCEAN SPRAY","ORGANIC VALLEY"
                       ,"PARMALAT","PETIT","R W KNUDSEN","RICE DREAM","ROBINSONS","SILK","SIR BANANAS","SNAPPLE"
                       ,"SO DELICIOUS","SSIPS","STARBUCKS","SUNCHY","SUNNY D","SUNSWEET","TAMPICO","TANTILLO"
                       ,"TEXSUN","TREE TOP","TROPICANA","TRU MOO","TUM-E YUMMIES","V8","VITA COCO"
                       ,"VOLCANO LEMON BURST","VOLCANO LIME BURST","WELCH'S","WYLER'S","YOO HOO","ZESTY LEMON"
                       ,"ZESTY LIME","CAPRI SUN","KOOL-AID","TANG","RET BR"]},
        {
            "Category" : "POWDERED BEV AND LIQ CONC",
            "package_ratio" : 1.0,
            "price" : [7.50, 9.99],
            "lb" : [1, 2],
            "Brand" : ["4C","AO JEL SERT BRND","ARIZONA","CRUSH","DASANI","EL MEXICANO","EMER'GEN-C","FLAVOR AID"
                       ,"GATORADE","GOYA","HANSEN'S","HAWAIIAN PUNCH","ICEE","JUICY JUICE","KERNS","KLASS","LIPTON"
                       ,"LUZIANNE","MARGARITAVILLE","MINUTE MAID","MOTTS","NESTEA","OCEAN SPRAY","POWERADE"
                       ,"POWEREDGE","PROPEL","PURE VIA","R W KNUDSEN","ROBINSONS","SKINNY GIRL","SNAPPLE"
                       ,"SODA STREAM","STUR","SUNNY D","SWEET LEAF","TANG","TAZO","TUM-E YUMMIES","V8","WELCH'S"
                       ,"WYLER'S","ZUKO","COUNTRY TIME","CRYSTAL LIGHT","KOOL-AID","MIO","RET BR"]}
        ]


ncols = len(cols["names"])
ncols

17

In [4]:
dfile = open("mc_panel_data_sample_v2.csv", "w")
cidx = 0
for name in cols["names"]:
    print(name)    
    if cidx > 0:
        dfile.write(',')    
    dfile.write(name)
    cidx +=1 
dfile.write("\n")

Market
Retailer
Department
Category
Brand
Period (4we)
$
Lbs
Units
Avg_Items
%ACV
TDP
No Promo$
Prc Desc $
Disp w/o Feat $
Feat & Disp $
Feat w/o Disp $


1

## Generate Monte Carlo & Save to File

First iterate through each brand, year, then through each assets.

In [5]:
c_market_txt = master_market_list[0]
c_retailer_txt = master_retailer_list[0]
c_dpt_txt= master_dpt_list[0]

for c_cat_idx in range(len(category_brand_dic)):
    c_cat_txt = category_brand_dic[c_cat_idx]["Category"]
    
    for c_brand_txt in category_brand_dic[c_cat_idx]["Brand"]:
        
        for c_period_txt in all_dates:
            print("Generating data for " + c_brand_txt + ": \n")        
            c_row = str(c_market_txt) + ','
            c_row += str(c_retailer_txt) + ','
            c_row += str(c_dpt_txt) + ','
            c_row += str(c_cat_txt) + ','
            c_row += str(c_brand_txt) + ','
            c_row += str(c_period_txt)

            # Definition each columns:
            for c_col_idx in range(6, ncols):       # First six columns are strings.  Start generating at this point.       
                if cols["names"][c_col_idx] == "Avg_Items":   # Handle specific columns where we want build in relationships.
                    c_col_data = np.abs(np.random.normal(5, 3))
                elif cols["names"][c_col_idx] == "Units":
                    if c_cat_txt == "DRY COFFEE":
                        c_col_data = np.random.normal(2000, 200)
                    elif c_cat_txt == "KIDS SINGLE SERVE BEVERAGES":
                        c_col_data = np.random.normal(1000, 50)
                    elif c_cat_txt == "POWDERED BEV AND LIQ CONC":
                        c_col_data = np.random.normal(500, 50)
                    else:
                        c_col_data = -3333333.3   # Fill error value
                elif cols["names"][c_col_idx] == "TDP":
                    c_col_data =  -2222222.2 # Fill error value
                else:                    
                    if cols["type"][c_col_idx] == "num":
                        c_col_data = -1111111.1  # Fill error value
                    elif cols["type"][c_col_idx] == "perc":
                        c_col_data = np.random.uniform(low=0.0, high=1.0)  # default is low=0.0, high=1.0
                    else:
                        c_col_data = -1000000.0 # Fill error value
                        
                c_row += ',' + str(c_col_data)

            # Write this to row. 
            dfile.write(c_row + "\n")    

print("Initial Monte Carlo Generation Finished Successfully.")

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for BARNIE'S: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for C D M: 

Generating data for CAFE BUSTELO: 

Generating data for CAFE BUSTELO: 

Generating data for CAFE BUSTELO: 

Generating data for CAFE BUSTELO: 

Generating data for CAFE BUSTELO: 

Generating data for CAFE BUSTELO: 

Generating data for CAFE BUSTELO: 

Generating data 

Generating data for JUICY JUICE: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for JUMEX: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for KERNS: 

Generating data for LANGERS: 

Generating data for LANGERS: 

Generating data for LANGERS: 

Generating data for LANGERS: 

Generating data for LANGERS: 

Generating data for LANGERS: 

Generating data for LANGERS: 

Generating data for LANGERS: 

Generating data for L


Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SKINNY GIRL: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SNAPPLE: 

Generating data for SODA STREAM: 

Generating data for SODA STREAM: 

Generating data for SODA STREAM: 

Generating data for SODA STREAM: 

Generating data for SODA STREAM: 

Generating data for SODA STREAM:

Close File

In [6]:
dfile.close()

## Now bake in relationships.

In [7]:
gdata = pd.read_csv("mc_panel_data_sample_v2.csv", sep=",", lineterminator="\n")

In [8]:
gdata.head()

,Market,Retailer,Department,Category,Brand,Period (4we),$,Lbs,Units,Avg_Items,%ACV,TDP,No Promo$,Prc Desc $,Disp w/o Feat $,Feat & Disp $,Feat w/o Disp $
0,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,06/03/17,-1111111.1,-1111111.1,1956.077294,0.685653,0.837619,-2222222.2,-1111111.1,-1111111.1,-1111111.1,-1111111.1,-1111111.1
1,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,07/01/17,-1111111.1,-1111111.1,1819.817065,5.103111,0.852252,-2222222.2,-1111111.1,-1111111.1,-1111111.1,-1111111.1,-1111111.1
2,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,08/05/17,-1111111.1,-1111111.1,2003.314369,3.568610,0.812398,-2222222.2,-1111111.1,-1111111.1,-1111111.1,-1111111.1,-1111111.1
3,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,09/02/17,-1111111.1,-1111111.1,2096.579885,5.210842,0.945694,-2222222.2,-1111111.1,-1111111.1,-1111111.1,-1111111.1,-1111111.1
4,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,09/30/17,-1111111.1,-1111111.1,2430.234788,9.696346,0.385159,-2222222.2,-1111111.1,-1111111.1,-1111111.1,-1111111.1,-1111111.1


#### Insert baseline values for $ and Lbs

In [9]:
for c_cat_idx in range(len(category_brand_dic)):
    c_cat_txt = category_brand_dic[c_cat_idx]["Category"]
    c_package_ratio = category_brand_dic[c_cat_idx]["package_ratio"]
    
    c_price_array = category_brand_dic[c_cat_idx]["price"]
    c_lb_array = category_brand_dic[c_cat_idx]["lb"]
    c_packratio_array = np.array([c_package_ratio, 1-c_package_ratio])
    
    # Set a default for each category, and build in the relationships later.  This will be used to create the lbs and $ columns. 
    c_lb = np.dot(c_packratio_array, c_lb_array)       
    c_price = np.dot(c_packratio_array, c_price_array)

    print("Updating Category=", c_cat_txt, " c_lb=", c_lb, " c_price=", c_price)
    gdata.loc[(gdata['Category']==c_cat_txt),'Lbs'] = gdata.loc[(gdata['Category']==c_cat_txt),'Units'] * c_lb
    gdata.loc[(gdata['Category']==c_cat_txt),'$'] = gdata.loc[(gdata['Category']==c_cat_txt),'Units'] * c_price

Updating Category= DRY COFFEE  c_lb= 0.76  c_price= 8.496
Updating Category= KIDS SINGLE SERVE BEVERAGES  c_lb= 1.0  c_price= 7.5
Updating Category= POWDERED BEV AND LIQ CONC  c_lb= 1.0  c_price= 7.5


#### Create TDP = %ACV * AVG_ITEMS. 

In [10]:
gdata["TDP"] = gdata["%ACV"] * gdata["Avg_Items"]

#### Let's now bake in the No Promo Sales and Promo Sales Columns

First, assume that the split is 60\% No Promo versus Promo. 

Then, split the promo sales into the following:
   - Prc Desc Sales: 30 \%
   - Disp w/o Feat Sales: 30 \%
   - Feat & Disp Sales: 10 \%
   - Feat w/o Disp Sales: 30 \%

In [11]:
promo_ratio = 0.4
promo_cols_labels = ["Prc Desc $", "Disp w/o Feat $", "Feat & Disp $", "Feat w/o Disp $"]
promo_array = [0.3, 0.3, 0.1, 0.3]  # Assumes the order of the probabilites are in the same order as above.
promo_cov = np.eye(len(promo_array)) * 0.003  # Keep this constant small to avoid negative probabilities

In [12]:
nrows = gdata.shape[0]
tmp_probs = np.abs(np.random.multivariate_normal(promo_array, promo_cov, nrows))
tmp_probs

array([[0.21482963, 0.32050163, 0.16914487, 0.29365357],
       [0.29361573, 0.34091173, 0.11851093, 0.28383274],
       [0.34003756, 0.33702004, 0.07258586, 0.36306031],
       ...,
       [0.27735064, 0.28231286, 0.012874  , 0.30842295],
       [0.3485931 , 0.27536675, 0.05592859, 0.20585311],
       [0.24331862, 0.25179473, 0.06189069, 0.29576472]])

Check to see if the averaged probabilities are the same as specified. 

In [14]:
np.mean(tmp_probs, axis=0)

array([0.29925443, 0.30056404, 0.10290557, 0.29827357])

Now let's create each column:

In [15]:
for idx in range(len(promo_cols_labels)):
    c_col_txt = promo_cols_labels[idx]
    print(" Updating", c_col_txt)
    gdata[c_col_txt] = gdata["$"] * tmp_probs[:,idx] * promo_ratio
    
print(" Updating No_promo$")
gdata["No Promo$"] = gdata["$"] - np.sum(gdata[promo_cols_labels], axis=1)

 Updating Prc Desc $
 Updating Disp w/o Feat $
 Updating Feat & Disp $
 Updating Feat w/o Disp $
 Updating No_promo$


In [16]:
gdata.head()

,Market,Retailer,Department,Category,Brand,Period (4we),$,Lbs,Units,Avg_Items,%ACV,TDP,No Promo$,Prc Desc $,Disp w/o Feat $,Feat & Disp $,Feat w/o Disp $
0,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,06/03/17,16618.832686,1486.618743,1956.077294,0.685653,0.837619,0.574316,9983.732481,1428.087101,2130.545176,1124.396135,1952.071793
1,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,07/01/17,15461.165785,1383.060969,1819.817065,5.103111,0.852252,4.349136,9048.671253,1815.856569,2108.357101,732.926864,1755.353998
2,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,08/05/17,17020.158879,1522.518920,2003.314369,3.568610,0.812398,2.899133,9444.800921,2314.997292,2294.453867,494.169136,2471.737662
3,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,09/02/17,17812.542706,1593.400713,2096.579885,5.210842,0.945694,4.927861,9312.520414,2287.720219,2844.566658,1336.934345,2030.801070
4,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,09/30/17,20647.274763,1846.978439,2430.234788,9.696346,0.385159,3.734634,13171.286960,1910.068335,2614.845233,388.416303,2562.657932


Checking to make sure that the average percentage for each columns is indeed the specified percentages of the original sales.

In [17]:
for idx in range(len(promo_cols_labels)):
    print(promo_cols_labels[idx], " average perc. of $ =", np.mean(gdata[promo_cols_labels[idx]]/gdata["$"]))
    
print("No_promo$", " average perc. of $ =", np.mean(gdata["No Promo$"]/gdata["$"]))    

Prc Desc $  average perc. of $ = 0.11970177219448518
Disp w/o Feat $  average perc. of $ = 0.12022561587544695
Feat & Disp $  average perc. of $ = 0.04116222763618269
Feat w/o Disp $  average perc. of $ = 0.11930942984572808
No_promo$  average perc. of $ = 0.5996009544481571


Checking the total difference between each row's (a) sum of all sales columns and (b) sales.  If the number below is not a really small number, then there is an issue. 

In [18]:
sum(np.sum(gdata[["No Promo$", "Prc Desc $", "Disp w/o Feat $", "Feat & Disp $", "Feat w/o Disp $"]], axis=1)-gdata["$"])

1.255102688446641e-10

#### Let's create the YA variables.

First assume last year for ALL variables was 6\% less than this year.  

In [19]:
tmp_ncols = 17 # 17 variables up for the current year.
for idx in range(6, tmp_ncols):
    print("Create YA variables for", cols["names"][idx])
    gdata[cols["names"][idx]+"_YA"] = 0.94 * gdata[cols["names"][idx]]


Create YA variables for $
Create YA variables for Lbs
Create YA variables for Units
Create YA variables for Avg_Items
Create YA variables for %ACV
Create YA variables for TDP
Create YA variables for No Promo$
Create YA variables for Prc Desc $
Create YA variables for Disp w/o Feat $
Create YA variables for Feat & Disp $
Create YA variables for Feat w/o Disp $


Spot check to see if this year's sales is 6% greater than last year's.

In [20]:
(np.mean(gdata["$"]) - np.mean(gdata["$_YA"]))/np.mean(gdata["$_YA"])

0.06382978723404276

In [21]:
# Make sure KSSB as a category is down 3% in $, Units, Lbs from YA:
for c_col_txt in ["$", "Units", "Lbs"]:
    gdata.loc[(gdata['Category']=="KIDS SINGLE SERVE BEVERAGES"),c_col_txt + "_YA"] = 1.03 * gdata.loc[(gdata['Category']=="KIDS SINGLE SERVE BEVERAGES"),c_col_txt]

Now let's make sure Honest Kids, Motts, and Good 2 Grow, has actually grown by 8%.  

In [22]:
for idx in range(6, tmp_ncols):
    for cbrand_txt in ["HONEST KIDS", "MOTTS", "GOOD 2 GROW"]:
        gdata.loc[(gdata['Brand']==cbrand_txt),cols["names"][idx]] = gdata.loc[(gdata['Brand']==cbrand_txt),cols["names"][idx]] * 1.08

In [23]:
(np.mean(gdata.loc[(gdata['Brand']=="MOTTS"), "$"]) - np.mean(gdata.loc[(gdata['Brand']=="MOTTS"), "$_YA"]))/np.mean(gdata.loc[(gdata['Brand']=="MOTTS"), "$_YA"])

0.08003599035081983

Hawaiian Punch's Units are down by 4\%:  Units summed for the Hawaiian Punch is 4 \% lower than Units_YA.

In [24]:
cbrand_txt = "HAWAIIAN PUNCH"
gdata.loc[(gdata['Brand']==cbrand_txt),"Units"] = gdata.loc[(gdata['Brand']==cbrand_txt),"Units_YA"] * 0.96
gdata.loc[(gdata['Brand']==cbrand_txt),"$"] = gdata.loc[(gdata['Brand']==cbrand_txt),"$_YA"] * 0.94

Update data

In [25]:
gdata.to_csv("mc_panel_data_sample_v2.csv", index=False)

## Examine Data

In [26]:
gdata = pd.read_csv("mc_panel_data_sample_v2.csv", sep=",", lineterminator="\n")

In [27]:
gdata.head()

,Market,Retailer,Department,Category,Brand,Period (4we),$,Lbs,Units,Avg_Items,...,Lbs_YA,Units_YA,Avg_Items_YA,%ACV_YA,TDP_YA,No Promo$_YA,Prc Desc $_YA,Disp w/o Feat $_YA,Feat & Disp $_YA,Feat w/o Disp $_YA
0,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,06/03/17,16618.832686,1486.618743,1956.077294,0.685653,...,1397.421619,1838.712656,0.644514,0.787362,0.539857,9384.708532,1342.401875,2002.712466,1056.932367,1834.947486
1,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,07/01/17,15461.165785,1383.060969,1819.817065,5.103111,...,1300.077311,1710.628041,4.796924,0.801117,4.088187,8505.750978,1706.905174,1981.855675,688.951252,1650.032758
2,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,08/05/17,17020.158879,1522.518920,2003.314369,3.568610,...,1431.167785,1883.115507,3.354494,0.763654,2.725185,8878.112866,2176.097455,2156.786635,464.518988,2323.433403
3,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,09/02/17,17812.542706,1593.400713,2096.579885,5.210842,...,1497.796670,1970.785092,4.898191,0.888952,4.632190,8753.769190,2150.457005,2673.892658,1256.718285,1908.953006
4,Total US,All Outlets,BEVERAGES,DRY COFFEE,BARNIE'S,09/30/17,20647.274763,1846.978439,2430.234788,9.696346,...,1736.159733,2284.420701,9.114565,0.362049,3.510556,12381.009742,1795.464235,2457.954519,365.111325,2408.898456


### Checking Data Assumptions:

1. Are there any values of AVG_ITEMS that are less than zero?

In [28]:
sum(gdata["Avg_Items"] < 0)

0

2. What is the average and sd units of the three categories?  Do they check out with the original DGP?

In [29]:
print("Mean=", gdata.loc[(gdata['Category']=="DRY COFFEE"),'Units'].mean())
print("SD=", gdata.loc[(gdata['Category']=="DRY COFFEE"),'Units'].std())

print("Mean=", gdata.loc[(gdata['Category']=="KIDS SINGLE SERVE BEVERAGES"),'Units'].mean())
print("SD=", gdata.loc[(gdata['Category']=="KIDS SINGLE SERVE BEVERAGES"),'Units'].std())

print("Mean=", gdata.loc[(gdata['Category']=="POWDERED BEV AND LIQ CONC"),'Units'].mean())
print("SD=", gdata.loc[(gdata['Category']=="POWDERED BEV AND LIQ CONC"),'Units'].std())

Mean= 2004.004108264904
SD= 202.66131462055242
Mean= 1001.7246554652841
SD= 52.982658202112056
Mean= 496.70602793807666
SD= 52.01032035292031


3. What is the average lbs for each categories?

4. What is the average $ for each category?

Is Hawaiian Punch's Sales down this year?

In [30]:
cbrand_txt = "HAWAIIAN PUNCH"
(np.mean(gdata.loc[(gdata['Brand']==cbrand_txt), "$"]) - np.mean(gdata.loc[(gdata['Brand']==cbrand_txt), "$_YA"]))/np.mean(gdata.loc[(gdata['Brand']==cbrand_txt), "$_YA"])

-0.060000000000000005

What about units?

In [31]:
(np.mean(gdata.loc[(gdata['Brand']==cbrand_txt), "Units"]) - np.mean(gdata.loc[(gdata['Brand']==cbrand_txt), "Units_YA"]))/np.mean(gdata.loc[(gdata['Brand']==cbrand_txt), "Units_YA"])

-0.039999999999999904